In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15173006
paper_name = 'wu_brown_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/inline-supplementary-material-1.txt', sep='\t')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4297 x 55


In [7]:
original_data.head()

,20040824,NAME,WEIGHT,DESC,AAPO,YPD,Wort,RafA,Lova,HygB,...,SC,Sorb-5g,Sorb-15g,Thr,Tpt,TPZ,Trp,UVA,UVB,UVC
0,YML095C,RAD10,1,removal of nonhomologous ends*,-0.480,0.014,-0.233,-0.137,0.035,0.063,...,-0.210,-0.085,-0.152,-0.211,-0.045,-1.540,-0.171,-0.248,-3.200,-3.082
1,YER162C,RAD4,1,"nucleotide-excision repair, DNA damage recogni...",-0.091,0.047,-0.059,0.052,0.054,0.106,...,-0.220,-0.093,-0.042,-0.201,0.128,-1.301,-0.239,-0.230,-3.296,-3.205
2,YLR032W,RAD5,1,DNA repair,-0.134,-0.544,-0.105,0.047,0.117,-0.257,...,-0.039,0.327,0.388,0.233,-0.046,-1.182,-0.074,-0.332,-2.529,-2.556
3,YMR201C,RAD14,1,"nucleotide-excision repair, DNA damage recogni...",-0.242,0.085,0.005,0.034,-0.078,-0.034,...,-0.234,-0.136,-0.224,-0.401,-0.208,-1.135,-0.220,0.178,-2.912,-2.762
4,YGR258C,RAD2,1,"nucleotide-excision repair, DNA incision, 3'-t...",-0.044,0.016,0.080,-0.060,0.131,0.047,...,-0.115,-0.066,0.007,-0.089,0.087,-0.675,-0.100,-0.098,-3.555,-3.417


In [8]:
original_data['orf'] = original_data['20040824'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [20040824, NAME, WEIGHT, DESC, AAPO, YPD, Wort, RafA, Lova, HygB, GlyE, Calc, Caff, AntA, ActD, Alk-5g, Alk-15g, AraC, Arn, Ars, Ben, Bleo, Cis1, Cis4, CPTa, CPTc, Dox, Gal-5g, Gal-15g, H2O2, HU, Ida, IR, Lys, Mech, Mel, Min-5g, Min-15g, MMC, MMS, NaCl-5g, NaCl-15g, Nys-5g, Nys-15g, Oxa, SC, Sorb-5g, Sorb-15g, Thr, Tpt, TPZ, Trp, UVA, UVB, UVC, orf]
Index: []

[0 rows x 56 columns]


In [12]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data.drop(columns = ['20040824','NAME','WEIGHT','DESC'], inplace=True)

In [15]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(4258, 51)

In [33]:
original_data.columns = original_data.columns.str.upper()

In [34]:
original_data.head()

,AAPO,YPD,WORT,RAFA,LOVA,HYGB,GLYE,CALC,CAFF,ANTA,...,SC,SORB-5G,SORB-15G,THR,TPT,TPZ,TRP,UVA,UVB,UVC
orf,,,,,,,,,,,,,,,,,,,,,
YAL002W,-1.347,-0.308,0.117,-0.425,-0.327,-1.672,-0.287,-1.268,-0.663,-0.001,...,0.190,0.500,0.391,0.058,-0.035,-0.920,0.409,-0.251,-0.013,0.106
YAL004W,-0.167,0.052,-0.102,-0.144,-0.144,-0.055,-0.003,0.110,0.079,0.205,...,0.139,0.139,0.119,0.161,-0.046,0.054,0.200,-0.063,-0.049,-0.028
YAL005C,-0.299,0.127,-0.073,-0.227,-0.203,-0.227,-0.076,-0.198,-0.175,0.112,...,-0.206,0.029,0.184,-0.203,-0.123,0.016,-0.057,0.212,-0.074,-0.055
YAL007C,-0.435,0.146,-0.137,-0.169,-0.194,-0.188,-0.076,-0.191,-0.234,0.117,...,-0.136,0.045,0.070,0.010,0.017,0.053,-0.003,0.002,-0.008,-0.018
YAL008W,-0.126,0.048,-0.007,-0.031,-0.329,-0.025,-0.242,-0.144,-0.155,-0.012,...,-0.171,0.044,0.267,-0.031,-0.104,0.031,-0.069,0.034,-0.034,-0.028


# Load dataset ids

In [59]:
dt = pd.read_csv('extras/dataset_id_name.txt', sep='\t', header=None)

In [60]:
dt[1] = dt[1].str.upper()

In [61]:
original_data = original_data[dt[1].values]

In [62]:
original_data.head()

,CIS1,OXA,MMC,CIS4
orf,,,,
YAL002W,-0.188,-0.877,-0.032,-0.375
YAL004W,-0.022,-0.032,0.065,-0.112
YAL005C,-0.054,-0.043,-0.010,-0.067
YAL007C,-0.072,-0.049,-0.040,-0.077
YAL008W,-0.035,-0.121,-0.082,-0.141


In [64]:
dataset_ids = dt[0].values

# Prepare the final dataset

In [65]:
data = original_data.copy()

In [66]:
datasets = datasets.reindex(index=dataset_ids)

In [67]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [68]:
data.head()

dataset_id,489,4749,4750,4754
data_type,value,value,value,value
orf,,,,
YAL002W,-0.188,-0.877,-0.032,-0.375
YAL004W,-0.022,-0.032,0.065,-0.112
YAL005C,-0.054,-0.043,-0.010,-0.067
YAL007C,-0.072,-0.049,-0.040,-0.077
YAL008W,-0.035,-0.121,-0.082,-0.141


## Subset to the genes currently in SGD

In [69]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [70]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,489,4749,4750,4754
,data_type,value,value,value,value
gene_id,orf,,,,
2,YAL002W,-0.188,-0.877,-0.032,-0.375
1863,YAL004W,-0.022,-0.032,0.065,-0.112
4,YAL005C,-0.054,-0.043,-0.010,-0.067
5,YAL007C,-0.072,-0.049,-0.040,-0.077
6,YAL008W,-0.035,-0.121,-0.082,-0.141


# Normalize

In [71]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [72]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [73]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,489,4749,4750,4754,489,4749,4750,4754
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
2,YAL002W,-0.188,-0.877,-0.032,-0.375,-0.771768,-2.386752,-0.309212,-1.901084
1863,YAL004W,-0.022,-0.032,0.065,-0.112,-0.097164,-0.088147,0.529828,-0.568759
4,YAL005C,-0.054,-0.043,-0.010,-0.067,-0.227208,-0.118070,-0.118914,-0.340795
5,YAL007C,-0.072,-0.049,-0.040,-0.077,-0.300358,-0.134391,-0.378411,-0.391454
6,YAL008W,-0.035,-0.121,-0.082,-0.141,-0.149994,-0.330248,-0.741706,-0.715670


# Print out

In [74]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [75]:
from IO.save_data_to_db3 import *

In [76]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/4 [00:00<?, ?it/s]

Deleting all datasets for PMID 15173006...
Inserting the new data...


100%|██████████| 4/4 [00:24<00:00,  6.03s/it]

Updating the data_modified_on field...
